# MLE 최대 우도 추정
#https://datascienceschool.net/view-notebook/864a2cc43df44531be32e3fa48769501/
#https://ratsgo.github.io/statistics/2017/09/23/MLE/

In [21]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [30]:
total_epoch =100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128
X = tf.placeholder(tf.float32,[None,n_input]) #실제값
Z = tf.placeholder(tf.float32,[None,n_noise]) #노이즈값

In [31]:
#128x256
G_W1 = tf.Variable(tf.random_normal([n_noise,n_hidden],stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden,n_input],stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

D_W1 = tf.Variable(tf.random_normal([n_input,n_hidden],stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden,1],stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [32]:
# 생성기와 판별기
# cost function: 확률적 함수
#지금까지 사용한것은 MSE, ENTROPY(분류일때)
# KL - Divergence : GAN , VAE, - 분포의 차를 확인하는 함수 
#생성기 -> noise가 input데이터 ,cost function : KL-Divergence


def generator(noise_z): #noise : 128  x128
    hidden =tf.nn.relu(tf.matmul(noise_z,G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden,G_W2) + G_b2)
    return output

def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs,D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden,D_W2)+D_b2)
    return  output

def get_noise(batch_size,n_noise):
    return np.random.normal(size = ( batch_size , n_noise))

In [33]:
G = generator(Z) #노이즈 => 이미지 생성  
D_gene = discriminator(G) # 노이즈로 부터 생성된 이미지를 판별 _ 판별의 의미: 분포를 확인한다
D_real = discriminator(X) #실제 이미지의  분포를 확인


#로그가 붙어있는 이유는 : 확률값을 구하는 것인데 확률에 log를 씌어주면 정보량 이다. 
#확률이 높아지면 정보량이 작아지고
#확률이 낮아지면 정보량이 커짐  : tfidf의 논리를 고려할것
#가장 적합한 분포 => 높은 것 유리 
loss_D = tf.reduce_mean(tf.log(D_real) +tf.log(1-D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene)) #최대우도 추정법() - 적합한 분포를 찾아내는 것- 확률이 가장 높은 것 

D_var_list = [D_W1,D_b1,D_W2,D_b2]
G_var_list = [G_W1,G_b1,G_W2,G_b2]
#최대화하기위해 -   #loss 에 마이너스 붙인것은 큰것을 찾아야하기때문
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,var_list = D_var_list) #역전파 변수를 지정 
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,var_list = G_var_list)
%matplotlib inline
sess = tf.Session()

sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D,loss_val_G =0,0

In [34]:
#  이미지 분포와  특징을 출출하는 망이다 
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size,n_noise) #n noise 128  줬다 위에서
        #실제 이미지 분포 
        _,loss_val_D = sess.run([train_D,loss_D],feed_dict ={X:batch_xs, Z:noise})
        # 이미지 생성 128x128
        _,loss_val_G = sess.run([train_G,loss_G],feed_dict = {Z: noise})
        
    print('Epoch :','%04d' % epoch, 
         'D loss:{:4}'.format(loss_val_D),
         'G loss: {:4}'.format(loss_val_G))
    if epoch ==0 or (epoch+1)%10==0:
        sample_size = 10 
        #노이즈로부터 이미지 생성 : 가중치곱해서 나온 generatoir가 이미지 특징을 잡고있다가 노이즈로부터 생성
        noise = get_noise(sample_size,n_noise) #10 x128
        samples = sess.run(G,feed_dict={Z:noise})
        fig,ax = plt.subplots(1,sample_size,figsize = (sample_size,1))
        for i in range(sample_size):
            ax[i].set_axis_off() # 128 : 784이미지 사이즈로 생성
            ax[i].imshow(np.reshape(samples[i],(28,28))) #샘플스를 28x28로  
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),
                   bbpx_inches ='tight')
        plt.close(fig)
print('최적화 완료')

Epoch : 0000 D loss:-0.3639432191848755 G loss: -2.1651804447174072
Epoch : 0001 D loss:-0.3383620083332062 G loss: -2.353578567504883
Epoch : 0002 D loss:-0.1635240912437439 G loss: -2.9644699096679688
Epoch : 0003 D loss:-0.36817944049835205 G loss: -1.8570829629898071
Epoch : 0004 D loss:-0.37713128328323364 G loss: -2.0835986137390137
Epoch : 0005 D loss:-0.32829391956329346 G loss: -2.2202770709991455
Epoch : 0006 D loss:-0.20435276627540588 G loss: -2.4840598106384277
Epoch : 0007 D loss:-0.22250375151634216 G loss: -2.912736177444458
Epoch : 0008 D loss:-0.5225299596786499 G loss: -2.3292386531829834
Epoch : 0009 D loss:-0.5562026500701904 G loss: -1.9747810363769531
Epoch : 0010 D loss:-0.29384762048721313 G loss: -2.561185598373413
Epoch : 0011 D loss:-0.3025922477245331 G loss: -2.3101766109466553
Epoch : 0012 D loss:-0.27663683891296387 G loss: -2.9050564765930176
Epoch : 0013 D loss:-0.3629705011844635 G loss: -2.6720361709594727
Epoch : 0014 D loss:-0.3529299199581146 G lo

In [ ]:

#  